# Bot UserName Telegram
## @ServePro_bot

In [2]:
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters
import pandas as pd
import nest_asyncio
import joblib
import os
import asyncio
import re

In [3]:
# Apply nest_asyncio for Jupyter Notebook compatibility
nest_asyncio.apply()

In [4]:
# Load the trained model and data
knn_model = joblib.load("knn_model.pkl")
data = pd.read_csv("Final_Merged_Data.csv")

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
# Clean up column names
data.columns = [col.strip() for col in data.columns]

In [6]:
# Start command
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(
        "Hello! Send me the name of a city and the type of service (e.g., 'Brampton, Electricians'), and I'll provide you with the top 5 service providers in that area!"
    )

In [7]:
# Handle user input to provide top 5 service provider suggestions
async def handle_city_service(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    try:
        user_input = update.message.text.strip().lower()
        
        # Expecting input in the format: city, service_type
        if ',' not in user_input:
            await update.message.reply_text("Please provide input in the format: city, service_type (e.g., 'Brampton, Electricians')")
            return
        
        city, service_type = [x.strip() for x in user_input.split(',', 1)]
        
        # Filter dataset by location and service type
        filtered_data = data[(data['Location'].str.lower() == city) & (data['Services Provided'].str.lower().str.contains(service_type))]
        
        if filtered_data.empty:
            await update.message.reply_text(f"Sorry, I couldn't find any {service_type} service providers in {city}. Please try another city or service type.")
        else:
            # Track the number of providers already shown
            context.user_data.setdefault('offset', 0)
            offset = context.user_data['offset']
            
            # Sort by rating and select next 5 providers
            top_providers = filtered_data.sort_values(by='Rating', ascending=False).iloc[offset:offset+5]
            
            if top_providers.empty:
                await update.message.reply_text("No more service providers to show. Send 'more' if you want to see more options.")
                return
            
            response = f"Here are the top {service_type} service providers in {city.capitalize()}:\n"
            for _, row in top_providers.iterrows():
                response += f"- {row['Name']} (Rating: {row['Rating']})\n  Address: {row['Address']}\n  Phone: {row['Phone Number']}\n  Website: {row['Website']}\n\n"
            await update.message.reply_text(response)
            
            # Update the offset to show more providers next time
            context.user_data['offset'] += 5
            await update.message.reply_text("If you want to see more service providers, please type 'more'.")
    except Exception as e:
        await update.message.reply_text(f"An error occurred: {e}")

In [ ]:
# Main function to run the bot
async def main():
    # Use your bot token here
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN", "7829481199:AAHqIdl2K4BrpCOWcEqjcv6Zt9z-t2-sBHw")
    application = Application.builder().token(bot_token).build()

    # Add command handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_city_service))

    # Run the bot
    await application.run_polling()

# Run the bot in Jupyter Notebook or as a script
await main()